# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [18]:
import urllib.request
import json
import os
import ssl
import pandas as pd

from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [19]:
ws = Workspace.from_config()
experiment_name = 'hyperparameter-experiment'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [22]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = BayesianParameterSampling({
      '--C': uniform(0.2, 5),
      '--max_iter': choice(100, 150, 200, 250, 300, 400)
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory = './', 
                entry_script = 'train_hf.py', 
                compute_target = "heart-f-cluster",
                pip_packages = ["pandas"]
                )


hyperdrive_run_config = HyperDriveConfig(estimator = estimator,
                                     hyperparameter_sampling = param_sampling,
                                     primary_metric_name = "Accuracy",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 40,
                                     max_concurrent_runs = 3)

In [24]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_dbe8b15a-5743-4288-ad92-484ea896d995
Web View: https://ml.azure.com/runs/HD_dbe8b15a-5743-4288-ad92-484ea896d995?wsid=/subscriptions/2a3b9c06-13fd-4499-8d62-0323ea7c8399/resourcegroups/networkwatcherrg/workspaces/ml-workspace&tid=1877b446-2ff7-4a58-b640-4d9c006695a8

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-23T21:44:41.685497][API][INFO]Experiment created<END>\n""<START>[2021-08-23T21:44:42.076183][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2021-08-23T21:44:42.219870][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_dbe8b15a-5743-4288-ad92-484ea896d995
Web View: https://ml.azure.com/runs/HD_dbe8b15a-5743-4288-ad92-484ea896d995?wsid=/subscriptions/2a3b9c06-13fd-4499-8d62-0323ea7c8399/resourcegroups/networkwatcherrg/workspaces/ml-workspace&tid=1877b446-2ff7-4a58-b640-4d9c006695a8



{'runId': 'HD_dbe8b15a-5743-4288-ad92-484ea896d995',
 'target': 'heart-f-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-23T21:44:41.433191Z',
 'endTimeUtc': '2021-08-23T22:22:45.473772Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '3826d1c4-e4ee-40f6-9760-065f6c83b4aa',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.8333333333333334',
  'best_child_run_id': 'HD_dbe8b15a-5743-4288-ad92-484ea896d995_12',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlworkspace5506950682.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_dbe8b15a-5743-4288-ad92-484ea896d995/azureml-logs/hyperdrive.txt?sv=201

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [25]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [26]:
import joblib

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Parameter Values:', parameter_values)

Best Run Id:  HD_dbe8b15a-5743-4288-ad92-484ea896d995_12
Accuracy: 0.8333333333333334
Parameter Values: ['--C', '2.9836277484604428', '--max_iter', '200']


In [28]:
#TODO: Save the best model
joblib.dump(value=best_run.get_details()['runDefinition']['arguments'], 
            filename=os.path.join('outputs', 'best_hyperdrive_model.joblib'))

['outputs/best_hyperdrive_model.joblib']

In [29]:
best_run.get_details()

{'runId': 'HD_dbe8b15a-5743-4288-ad92-484ea896d995_12',
 'target': 'heart-f-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-23T21:55:55.213759Z',
 'endTimeUtc': '2021-08-23T21:58:20.910519Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '3826d1c4-e4ee-40f6-9760-065f6c83b4aa',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'd23a1d65-081c-43d2-8541-dcfc67dedc3e'}, 'consumptionDetails': {'type': 'Reference'}}],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_hf.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--C', '2.9836277484604428', '--max_iter', '200'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'heart-f-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': None,
  'no

## For the deployment part, please refer to the AutoML jupyter notebook.